# Lab 6: Reactive UIs and API Access
*Due 5 PM, Tuesday March 19, 2019*


The goal of this lab is to build upon your work in the previous week to wire up the app to the web service you built a couple of weeks ago. First, we will modify the todo app for users. Then we will replace the operations on the local array with functions that access the remote API.

## Problem 1: List Users
*10 Points*

Create a store called `UsersStore`, modeled loosely after `TodoStore`. You can store your users as an array of objects for now. Initialize the array with a couple of dummy users to get started.

Now create a listing of users in a `table` element. You should modify the existing `TodoView`.


## Problem 2: Create a Form to Create Users
*10 Points*

The form shall have fields for the user first and last name, email address, and select box for role. All fields are required and the email address should be validated for uniqueness. Display validation errors in a box below the form. (Look at [validatejs](https://validatejs.org) or another similar library to simplify validation. And write the validation part only after the basic form is working.)


## Problem 3: Modify the User Store to Fetch from an API
*10 Points*

We'll now point our app at a remote API, the one you wrote in Flask for Lab 3.

To do this, we'll use the Javascript [`fetch`](https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API/Using_Fetch) function. The basics of `fetch` are covered in class. However, it's important to understand that fetch is *asynchronous*. Instead of pausing the caller and waiting for results to be delivered, fetch returns immediately after sending out the request. What fetch returns is a [**Promise**](https://javascript.info/promise-basics). A promise is simply an object that can be supplied with various callback functions, actions to take when the result does arrive or when an error occurs. Here's an example:

```javascript
    
    const fetchUsers = () => {
        const fetchPromise = fetch("localhost:3000/users")
        # at this point all we have is a promise, not yet the result
    
        # Register a `then` callback to handle the result when
        # the promise 'resolves'
        fetchPromise.then(data => console.log(JSON.stringify(data)))

        # Register a `catch` callback to handle an error
        fetchPromise.catch(error => console.log(error))
        
        # return the promise
        return fetchPromise
        
    }
    
```

In typical usage, any call to `fetchUsers` will complete well before the fetch operation completes. So it is the job of the `then` and `catch` handlers to take care of handling data and errors. How to handle the data with mobx and react is explained below.

Also, in typical usage, one wouldn't separate the fetch operation from the registration of the `then` and `catch` handlers as above. (The above formulation does help explain that a Promise is just an object with operations on it.)

Instead, you would *chain* `fetch`, `then`, and `catch` into a single expression, like this:

```javascript
    const fetchUsers = () => {
        return fetch("localhost:3000/users").
            then(data => console.log(JSON.stringify(data))).
            catch(error => console.log(error))
    }
```

You can do this because `fetch`, `then`, and `catch` all return the same updated promise.

With this very basic understanding, you are now in a position to retrieve users from the API. We'll proceed in steps:

1. Add an observable instance variable called `fetchState` and initialize it to `'idle'`. You will use this to track the state of API fetch operations.

1. Add a method called `fetchUsers` to `UsersStore`. This method will fetch all the users from the API as shown above. The `then` callback should take the returned data and assign it to the observable instance variable that holds the users list. You should wrap the `then` callback function in a mobx `action(...)` decorator. This is necessary because the callback will be invoked asynchronously and mobx needs to be told that the function is changing observable state. See [here](https://mobx.js.org/best/actions.html) for a detailed rationale and also many examples of using `fetch`. Also, set the `fetchState` to 'pending' before issuing the fetch request and to 'done' in the `then` callback. You can set the `fetchState` to 'error' in the `catch` handler.

1. The last step is to tell the React UI to fetch users from the API when it's top-level component loads. To do this, add a `componentDidMount` lifecycle method to your top-level React component and call `fetchUsers` from the `usersStore`. 

## Problem 4: Track API Fetch State

Add an area (e.g., a `<div>`) to the app. This area will track the `fetchState` observable variable you defined in the previous problem.

## Problem 5: Create a User

*10 Points*

This problem follows a similar form as the previous problem, except you will modify the  `createUser` method in the User store to `POST` to `/users`. To have fetch send a `POST` instead of the default `GET`, you specify a second argument to the fetch call, like this:

```javascript
fetch(url, {
    method: 'post',
    body: JSON.stringify(newUser)
})
```

Note that the body of the POST is also attached to the same object that specifies the method.

## Problem 6: Edit Users
*30 points*

This problem is more involved than the above. Accordingly, it is worth more points.

1. Add an "Edit" button to each row of the user listing table. The handler for each button will call a method called `setEditingUser`, an *action* on the Users store. This method will take an ID as argument and will retrieve the specified user from the users store array and save a copy in an observable `editingUser` instance variable. You will use this observable to communicate with the UI form whether a user is being created or edited. That is, set `editingUser` to a blank user with a null ID if a user is being created and to the specified user if being edited.

2. Modify the form to react to changes in `editingUser`. Essentially, use the value of in `editingUser` to keep the form populated.

2. Create an `updateUser` method in the Users store. This method will take a User object (`{id = ..., email: ...,}`) and issue a PATCH or PUT to `users/{id}`.

2. Create a `saveUser` method in the User store. This will take the value in `editingUser` and either update the user or create a new one depending on whether the object has an ID set or not.

2. On successful update or create, the form should be reset to a new blank user.


## Homework Submission

Please submit a link to your forked and modified codesandbox app to Gradescope. Precise instructions to follow.